## Code and methods for data preprocessing.

Topics:

1. Finding and removing outliers in the data.
2. Finding and removing gaps in the data.
3. Resampling the data.

In [2]:
# Important necessary libraries

from typing import List, Tuple
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import datetime
from scipy import signal
import warnings

#ignore warnings for the notebook
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

In [37]:
# read data as a dataframe

df = pd.read_csv('02336490_raw_data_manual.csv')

df.head()

<bound method NDFrame.head of         Unnamed: 0                 DATE  gauge_height
0                0  2008-01-01 00:00:00          5.76
1                1  2008-01-01 00:15:00          5.75
2                2  2008-01-01 00:30:00          5.73
3                3  2008-01-01 00:45:00          5.71
4                4  2008-01-01 01:00:00          5.69
...            ...                  ...           ...
571181      571181  2024-05-29 07:15:00          6.36
571182      571182  2024-05-29 07:30:00          6.36
571183      571183  2024-05-29 07:45:00          6.35
571184      571184  2024-05-29 08:15:00          6.35
571185      571185  2024-05-29 08:30:00          6.34

[571186 rows x 3 columns]>

## Finding Errors in the Data

In some cases, there may be errors in the data you obtained.
Sources of errors can include data entry mistakes, anomalies due to faulty sensors, etc.

In our case, we only selected data with an 'A' approval mark, so there is a low chance of finding any errors. Nevertheless, we will review how to identify errors if they exist.

We can use the `describe()` method to view basic statistics such as minimum value, maximum value, median, and the 1st and 3rd quartiles. Generally, errors have values that are either extremely high, extremely low, or negative.  
These can be identified using this method.

**If errors are found in the data, we simply remove them and then continue with the rest of the preprocessing methods.**

In [38]:
df.describe()

,Unnamed: 0,gauge_height
count,571186.000000,571186.000000
mean,285592.500000,6.574300
std,164887.339766,2.980596
min,0.000000,2.790000
25%,142796.250000,4.300000
50%,285592.500000,5.480000
75%,428388.750000,7.920000
max,571185.000000,27.860000


- We don’t see any anomalous results in the statistics above, which suggests there are no notable errors.
- However, the maximum value is significantly greater than the mean and isn’t captured within the standard deviation range.
- Let’s check how many values are greater than 20.

In [39]:
(df['gauge_height'] > 20.0).sum()

## 1665 is a lot of values. We conclude that there's no error in the data.

np.int64(1665)

## Finding Gaps in the Data

Generally, two types of gaps can be found in time series data:

1. Since hourly data is recorded at 15-minute intervals, there is a chance that an entire row might be missing, resulting in gaps in the time sequence itself.
2. The second type occurs when the timestamp is present, but the gauge height is missing.

We need to address the first type of gap and then proceed to the second.

The following method will print all the time gaps in the data.

In [40]:
def find_missing_dates(df: pd.DataFrame, date_col: str = 'DATE', 
                       threshold: datetime.timedelta = datetime.timedelta(minutes=15)) -> Tuple[datetime.timedelta, datetime.timedelta]:
    """
    Find gaps in the date column of a DataFrame.
    
    Args:
    df (pd.DataFrame): Input DataFrame.
    date_col (str): Name of the date column.
    threshold (datetime.timedelta): Threshold for considering a gap.
    
    Returns:
    Tuple[datetime.timedelta, datetime.timedelta]: Maximum gap and total gap.
    """

    # convert the DATE column to datetime datatype - originally string
    df[date_col] = pd.to_datetime(df[date_col])

    # sort values in ascending order by DATE column
    df = df.sort_values(by=date_col)
    
    prev = df[date_col].iloc[0]
    
    total_gap = datetime.timedelta(minutes=0)
    max_diff = threshold

    print_gaps = 20
    total_gaps = 0

    print(f'Printing first {print_gaps} gaps...\n')
    for d in df[date_col].iloc[1:]:
        diff = d - prev
        if diff > max_diff:
            max_diff = diff
        if diff > threshold:
            total_gap += diff
            if print_gaps > 0:
                print(f"Gap found between {prev} and {d}: Gap: {diff}")
            print_gaps -= 1
            total_gaps += 1
        prev = d

    print(f'\nTotal number of gaps: {total_gaps}')
    print(f'Max gap: {max_diff}')
    print(f'Total gap: {total_gap}')
    return max_diff, total_gap


find_missing_dates(df)

Printing first 20 gaps...

Gap found between 2008-03-09 01:45:00 and 2008-03-09 03:00:00: Gap: 0 days 01:15:00
Gap found between 2008-03-14 08:00:00 and 2008-03-14 09:00:00: Gap: 0 days 01:00:00
Gap found between 2008-03-20 19:15:00 and 2008-03-20 20:30:00: Gap: 0 days 01:15:00
Gap found between 2008-03-22 07:00:00 and 2008-03-22 07:45:00: Gap: 0 days 00:45:00
Gap found between 2008-03-23 07:00:00 and 2008-03-23 07:45:00: Gap: 0 days 00:45:00
Gap found between 2008-03-24 03:45:00 and 2008-03-24 04:15:00: Gap: 0 days 00:30:00
Gap found between 2008-03-31 07:00:00 and 2008-03-31 07:30:00: Gap: 0 days 00:30:00
Gap found between 2008-04-01 16:00:00 and 2008-04-01 17:15:00: Gap: 0 days 01:15:00
Gap found between 2008-04-03 07:15:00 and 2008-04-03 07:45:00: Gap: 0 days 00:30:00
Gap found between 2008-04-03 08:15:00 and 2008-04-03 08:45:00: Gap: 0 days 00:30:00
Gap found between 2008-04-03 18:30:00 and 2008-04-03 19:00:00: Gap: 0 days 00:30:00
Gap found between 2008-04-04 07:00:00 and 2008-04

(Timedelta('5 days 00:15:00'), Timedelta('49 days 05:30:00'))

## Filling Missing Dates

This process involves identifying the minimum and maximum dates in the dataset, then generating a complete datetime range at the specified frequency within that interval.  
A new column is created for each timestamp in this range, and corresponding values from the original data are assigned to matching timestamps.

Datetimes with no matching data will remain as empty entries.

In [41]:

def fill_missing_dates(df: pd.DataFrame, freq = '15min' ) -> pd.DataFrame:
    """
    Process USGS data: set index, fill missing values, and interpolate.
    
    Args:
    df (pd.DataFrame): Input DataFrame with USGS data.
    
    Returns:
    pd.DataFrame: Processed DataFrame.
    """
    df.set_index('DATE', inplace=True)
    df.sort_index(inplace=True)
    df = df.groupby(df.index).mean()

    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
    df_filled = df.reindex(full_range)
    
    df_filled.reset_index(inplace=True)
    df_filled.rename(columns={'index': 'DATE'}, inplace=True)
    return df_filled

df = fill_missing_dates(df)


## Missing Values

In this section, we will discuss methods to fill missing values.

First, let’s calculate the percentage of missing values.

In [42]:
missing_count = df['gauge_height'].isnull().sum()
print(f'Percentage of missing values: {missing_count / len(df) * 100} %')

Percentage of missing values: 0.7372736863510514 %


Usually, less than 5% missing values is acceptable in a dataset.  
In our case, it’s less than 1%, which is within an acceptable range.  
The size of gaps also matters, as larger gaps are generally less favorable.

**Let’s examine the size of gaps in our data.**

In [43]:
import math

gap_sizes  = []
gap_size = 0
for val in df['gauge_height']:
    if math.isnan(val):
        gap_size += 1
    else:
        if gap_size > 0:
            gap_sizes.append(gap_size)
            gap_size = 0

# sort in descending order
sorted_gaps = sorted(gap_sizes, reverse=True)
print(f'Top 15 gaps: {sorted_gaps[:15]}')
print('all gaps: ', sorted_gaps)

Top 15 gaps: [480, 480, 384, 384, 288, 192, 192, 192, 192, 142, 96, 96, 96, 96, 96]
all gaps:  [480, 480, 384, 384, 288, 192, 192, 192, 192, 142, 96, 96, 96, 96, 96, 46, 7, 7, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<hr>

We can see that the maximum gap is 480 entries, which translates to \( 480/4 = 120 \) hours, or 5 days. This is a large gap.  
Regardless of the filling method used, it’s unlikely to perform better than simple linear interpolation in this case.  
One option, when using machine learning models, is to load data in a way that skips these large gap periods.

For our case, however, we’ll proceed with linear interpolation since the percentage of gaps is very low relative to the overall data size. Thus, any data errors introduced will have a negligible impact on the model.

If you encounter many small gaps in your data, you could use techniques like the Kalman filter or other autoregressive methods to estimate the gaps more accurately.

Let’s use linear interpolation to fill in the gaps.

In [44]:
def fill_missing_values(data: pd.DataFrame, method='linear') -> pd.DataFrame:
    """
    Process data: interpolate missing values with linear interpolation.
    
    Args:
    data (pd.DataFrame): Input DataFrame with data and 'DATE' column, which will be used as index.
    
    Returns:
    pd.DataFrame: Processed DataFrame.
    """
    data.set_index('DATE', inplace=True)
    data.sort_index(inplace=True)
    data = data.groupby(data.index).mean()
    
    for col in data.columns:
        data[col] = data[col].interpolate(method = method)

    data.reset_index(inplace=True)
    data.rename(columns={'index': 'DATE'}, inplace=True)
    return data

filled_df = fill_missing_values(df)

**How does this affect the statistics? Let's see.**

In [45]:
print('Before filling gaps\n', df['gauge_height'].describe())
print('\n\nAfter filling gaps\n', filled_df['gauge_height'].describe())

Before filling gaps
 count    571121.000000
mean          6.574456
std           2.980629
min           2.790000
25%           4.300000
50%           5.480000
75%           7.920000
max          27.860000
Name: gauge_height, dtype: float64


After filling gaps
 count    575363.000000
mean          6.573636
std           2.978890
min           2.790000
25%           4.310000
50%           5.480000
75%           7.920000
max          27.860000
Name: gauge_height, dtype: float64


- We can see that there’s very little change in the basic statistics.

## Resampling Data

- The last step we need to address is data resampling.
- We are going to incorporate meteorological data, which is collected at 1-hour intervals.
- Since our data is at 15-minute intervals, we will resample it to a 1-hour interval by calculating the mean for all values within each hour.
- Pandas provides a `resample` method, which will handle this for us.

In [46]:
def resample_to_hourly(df: pd.DataFrame) -> pd.DataFrame:
    """
    Resample 15-minute data to hourly data.
    It resamples by calculating mean for each column, this method will not work for 
    something like precipitaion where we need to do sum instead.
    
    Args:
    df (pd.DataFrame): Input DataFrame with 15-minute data.
    value_column (str): Name of the column containing the values to be resampled.
    
    Returns:
    pd.DataFrame: Resampled DataFrame with hourly data.
    """
    
    df = df.set_index('DATE')
    df = df.sort_index()
    
    # Resample to hourly frequency, using the mean
    resampled = df.resample('h', label='left', closed='left').mean()
    
    # Reset the index to make the DATE a column again
    resampled.reset_index(inplace=True)
    resampled.rename(columns={'index': 'DATE'}, inplace=True)

    return resampled


resampled_df = resample_to_hourly(filled_df)

In [47]:
# finally we have the reampled data

print(resampled_df.head())

# This 'Unnamed: 0' column was originally index column i.e. count for each record. It's values got changed because of pre processing operations. We are going to remove it when saving the final output.

                 DATE  Unnamed: 0  gauge_height
0 2008-01-01 00:00:00         1.5        5.7375
1 2008-01-01 01:00:00         5.5        5.6450
2 2008-01-01 02:00:00         9.5        5.5425
3 2008-01-01 03:00:00        13.5        5.4575
4 2008-01-01 04:00:00        17.5        5.3625


## Meteorological Data

We retrieve the data from this link: <a href='https://www.ncei.noaa.gov/access/search/data-search/local-climatological-data'>https://www.ncei.noaa.gov/access/search/data-search/local-climatological-data</a>.

We obtain a CSV file with hourly data, which includes various columns. From these, we will select the following columns:

1. DATE
2. Wet Bulb Temperature
3. Dry Bulb Temperature
4. Precipitation
5. Relative Humidity
6. Wind Speed
7. Station Pressure

We will apply the same methods as above to this dataset, with small modifications.

Let’s start by reading the data into a dataframe.

In [48]:
data = pd.read_csv('Fulton_county_full.csv')

# Show all the columns in the csv file
data.columns

Index(['STATION', 'DATE', 'REPORT_TYPE', 'SOURCE', 'HourlyAltimeterSetting',
       'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyPresentWeatherType',
       'HourlyPressureChange', 'HourlyPressureTendency',
       'HourlyRelativeHumidity', 'HourlySeaLevelPressure',
       'HourlySkyConditions', 'HourlyStationPressure', 'HourlyVisibility',
       'HourlyWetBulbTemperature', 'HourlyWindSpeed'],
      dtype='object')

We will select a subset of columns as mentioned above.  
A dictionary has been created to shorten the column names for convenience.

In [49]:
name_mapper = {
        'DATE': 'DATE',
        'HourlyWetBulbTemperature': 'WetBulbTemp',
        'HourlyDryBulbTemperature': 'DryBulbTemp',
        'HourlyPrecipitation': 'Precip',
        'HourlyRelativeHumidity': 'RelHumidity',
        'HourlyWindSpeed': 'WindSpeed',
        'HourlyStationPressure': 'StationPressure'
    }

relevant_columns = name_mapper.keys()

# Only select the relevant columns and ignore the rest
data = data[relevant_columns]

# Rename the columns to shorter names for comfort
data.rename(columns=name_mapper, inplace=True)

In [50]:
# Let's look at the dataset
data = data[data['DATE'] > '2008-01-01']
data.head()

,DATE,WetBulbTemp,DryBulbTemp,Precip,RelHumidity,WindSpeed,StationPressure
22332,2008-01-01T00:53:00,37,38,0.00,89,0.0,29.19
22333,2008-01-01T01:53:00,38,39,0.00,89,5.0,29.18
22334,2008-01-01T02:53:00,38,40,0.00,86,5.0,29.18
22335,2008-01-01T03:53:00,38,40,0.00,86,6.0,29.19
22336,2008-01-01T04:53:00,41,45,0.00,68,11.0,29.21


In [51]:
# Let's count the NaN values in the data

print("Percentage of data that is missing")
data.isna().sum() / len(data)*100

Percentage of data that is missing


DATE                0.000000
WetBulbTemp         3.980353
DryBulbTemp         3.512513
Precip             17.174819
RelHumidity         3.711424
WindSpeed           3.711955
StationPressure     3.558660
dtype: float64

<hr>

We observe that 17% of the precipitation data is missing (NaN).

**Let’s examine this in the dataset.**

We find that many values are zero, some are missing, and some are marked as `'T'`.

What does `'T'` mean?

After some research, we find that in precipitation datasets, `'T'` usually stands for "Trace," indicating a very small amount of precipitation, typically around 0.01 inches or less.

Therefore, we will replace `'T'` values with a small value, 0.005.

In [52]:
data['Precip'] = data['Precip'].replace('T', 0.005)

First we will look at the date gaps like we did above.

In [53]:
# convert DATE column to datetime object

data['DATE'] = pd.to_datetime(data['DATE'], errors='coerce')

In [54]:
# Find missing dates using the method we wrote above

find_missing_dates(data, 'DATE', datetime.timedelta(hours=1))

Printing first 20 gaps...

Gap found between 2008-01-16 20:22:00 and 2008-01-16 21:53:00: Gap: 0 days 01:31:00
Gap found between 2008-01-27 09:50:00 and 2008-01-27 10:53:00: Gap: 0 days 01:03:00
Gap found between 2008-03-24 23:59:00 and 2008-03-25 02:53:00: Gap: 0 days 02:54:00
Gap found between 2008-07-31 18:53:00 and 2008-07-31 20:53:00: Gap: 0 days 02:00:00
Gap found between 2008-08-12 15:52:00 and 2008-08-12 16:53:00: Gap: 0 days 01:01:00
Gap found between 2008-11-18 08:52:00 and 2008-11-18 09:56:00: Gap: 0 days 01:04:00
Gap found between 2008-11-18 10:52:00 and 2008-11-18 11:53:00: Gap: 0 days 01:01:00
Gap found between 2009-07-24 11:53:00 and 2009-07-24 13:53:00: Gap: 0 days 02:00:00
Gap found between 2009-08-26 20:53:00 and 2009-08-26 23:59:00: Gap: 0 days 03:06:00
Gap found between 2009-09-08 12:51:00 and 2009-09-08 13:53:00: Gap: 0 days 01:02:00
Gap found between 2009-11-02 09:53:00 and 2009-11-02 11:53:00: Gap: 0 days 02:00:00
Gap found between 2009-11-02 18:53:00 and 2009-11

(Timedelta('0 days 04:54:00'), Timedelta('2 days 12:45:00'))

In [55]:
for col in data.columns:
    if col != 'DATE':
        # convert non numeric invalid values to NaN
        non_numeric_mask = pd.to_numeric(data[col], errors='coerce').isna() & data[col].notna()
        non_numeric_values = data.loc[non_numeric_mask, col].unique()
        if len(non_numeric_values) > 0:
            print(f"Non-numeric values in {col}:\n{non_numeric_values}")
        data[col] = pd.to_numeric(data[col], errors='coerce')

Non-numeric values in WetBulbTemp:
['*']
Non-numeric values in DryBulbTemp:
['*']
Non-numeric values in RelHumidity:
['*']
Non-numeric values in WindSpeed:
['*']


<hr>

We notice there aren’t many gaps in the data, but a few issues need addressing.

First, the data is recorded at various minutes within each hour, so the `DATE` column does not align perfectly with hour intervals. As a result, two consecutive records are not always exactly 1 hour apart.

We need to write code to correct this.

**Defining the Concept:**

When we say the precipitation at `2013-06-10 04:00:00` is 0.5 inches, it means the rain gauge height increased by 0.5 inches over the last hour due to rain.

So, to adjust our data, we will:
1. Loop through each record.
2. For a given hour `x`, sum the rainfall that occurred between `x-1` hour and `x` hour.  
3. For other columns, calculate the mean of values within that hour.

There is an efficient way to accomplish this using Pandas. But before proceeding, let’s ensure all columns are converted to numeric data types.

And then resample the data to hourly. Note that the following operation ignores the nan values.

In [56]:
# Add a column with only the hour component
data['DATE_hour'] = data['DATE'].dt.ceil('h')  # Rounds up to the nearest hour

# Group by the hourly timestamp, summing 'Precip' and averaging other columns
hourly_data = data.groupby('DATE_hour').agg({
    'Precip': 'sum',
    **{col: 'mean' for col in data.columns if col not in ['Precip', 'DATE', 'DATE_hour']}
}).reset_index()

# Rename DATE_hour back to DATE to match the original structure
hourly_data.rename(columns={'DATE_hour': 'DATE'}, inplace=True)

In [57]:
hourly_data

,DATE,Precip,WetBulbTemp,DryBulbTemp,RelHumidity,WindSpeed,StationPressure
0,2008-01-01 01:00:00,0.0,37.0,38.0,89.0,0.0,29.19
1,2008-01-01 02:00:00,0.0,38.0,39.0,89.0,5.0,29.18
2,2008-01-01 03:00:00,0.0,38.0,40.0,86.0,5.0,29.18
3,2008-01-01 04:00:00,0.0,38.0,40.0,86.0,6.0,29.19
4,2008-01-01 05:00:00,0.0,41.0,45.0,68.0,11.0,29.21
...,...,...,...,...,...,...,...
143785,2024-05-27 23:00:00,0.0,68.0,73.0,79.0,3.0,29.06
143786,2024-05-28 00:00:00,0.0,68.0,71.0,87.0,0.0,29.07
143787,2024-05-28 01:00:00,0.0,67.0,69.0,90.0,0.0,29.08
143788,2024-05-28 02:00:00,0.0,68.0,69.0,93.0,0.0,29.07


Now, let's fill up the date gaps in the data. We have the method we used previously.

In [58]:
## Let's fill the date gaps

hourly_data = fill_missing_dates(hourly_data, freq = '1h' )

In [59]:
hourly_data.isna().sum() / len(hourly_data) * 100

DATE               0.000000
Precip             0.014602
WetBulbTemp        0.734297
DryBulbTemp        0.310825
RelHumidity        0.438075
WindSpeed          0.476320
StationPressure    0.318474
dtype: float64

### Methods to fill the gap in the data
We now observe that there are very few gaps, with the percentage being less than 1%.

We can use different methods to fill the gap. Examples include 

- Forward Fill (ffill): This method fills a missing value with the last known non-missing value. This is useful for data where values tend to stay constant between observations.

- Backward Fill (bfill): This is the opposite of forward fill. It fills a missing value with the next known non-missing value. This method assumes the value at a missing point is more like what's about to happen than what has already happened.

- Mean Fill: This technique calculates the average (mean) of the entire time series and uses that single value to replace all missing data points. It's a very simple approach but can be problematic as it ignores any trends or seasonality and can artificially reduce the variance.

- Regression-based Imputation: It treats the variable with missing values as a target and uses other variables (or the time index itself) as predictors in a regression model. The model then predicts what the missing values should be. This can be very effective because it can capture the underlying trends and relationships in the data to make a more intelligent guess.

But in our case we will use linear interpolation to fill these gaps, as we did previously for gauge heights, it's similar to mean based method, but we compute mean locally.

Since we already implemented this function above, we can simply call it here.

In [60]:
filled_data = fill_missing_values(hourly_data)

In [61]:
filled_data.describe()

,DATE,Precip,WetBulbTemp,DryBulbTemp,RelHumidity,WindSpeed,StationPressure
count,143811,143811.000000,143811.000000,143811.000000,143811.000000,143811.000000,143811.000000
mean,2016-03-15 02:00:00,0.013038,56.219143,62.414728,69.734418,4.714649,29.173258
min,2008-01-01 01:00:00,0.000000,4.000000,6.000000,10.000000,0.000000,28.256667
25%,2012-02-07 01:30:00,0.000000,45.000000,50.000000,53.000000,0.000000,29.080000
50%,2016-03-15 02:00:00,0.000000,59.000000,64.000000,73.000000,5.000000,29.170000
75%,2020-04-21 02:30:00,0.000000,69.000000,75.000000,88.666667,7.500000,29.270000
max,2024-05-28 03:00:00,7.200000,81.000000,103.000000,100.000000,29.500000,29.840000
std,NaN,0.126359,14.438374,16.251127,20.956541,4.371269,0.154944


In [62]:
# Check for NaN values
filled_data.isna().sum() # Meterological data

DATE               0
Precip             0
WetBulbTemp        0
DryBulbTemp        0
RelHumidity        0
WindSpeed          0
StationPressure    0
dtype: int64

In [63]:
resampled_df.isna().sum() # Gauge height data

DATE            0
Unnamed: 0      0
gauge_height    0
dtype: int64

***In the next step, we will combine the dataframes and save them in a single file for future use.***

Below is the method to combine the dataframes. It uses the `DATE` column as the common column to merge them.

In [64]:
def combine_dataframes(df1, df2, common_col = 'DATE'):

    # set the common column 'DATE' as index
    df1.set_index(common_col, inplace=True)
    df2.set_index(common_col, inplace=True)

    # min_date is the maximum start date from both dataframes
    min_date = max(df1.index.min(), df2.index.min())

    # max_date is the minimum end date from both dataframes
    max_date = min(df1.index.max(), df2.index.max())

    # combine the dataframes
    df = pd.concat([df1, df2], axis = 1)
    
    df = df[min_date:max_date]
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'DATE'}, inplace=True)

    return df

In [65]:
combined_df = combine_dataframes(filled_data, resampled_df)

In [66]:
combined_df

,DATE,Precip,WetBulbTemp,DryBulbTemp,RelHumidity,WindSpeed,StationPressure,Unnamed: 0,gauge_height
0,2008-01-01 01:00:00,0.0,37.0,38.0,89.0,0.0,29.19,5.5,5.6450
1,2008-01-01 02:00:00,0.0,38.0,39.0,89.0,5.0,29.18,9.5,5.5425
2,2008-01-01 03:00:00,0.0,38.0,40.0,86.0,5.0,29.18,13.5,5.4575
3,2008-01-01 04:00:00,0.0,38.0,40.0,86.0,6.0,29.19,17.5,5.3625
4,2008-01-01 05:00:00,0.0,41.0,45.0,68.0,11.0,29.21,21.5,5.2175
...,...,...,...,...,...,...,...,...,...
143806,2024-05-27 23:00:00,0.0,68.0,73.0,79.0,3.0,29.06,571057.5,7.2175
143807,2024-05-28 00:00:00,0.0,68.0,71.0,87.0,0.0,29.07,571061.5,7.0725
143808,2024-05-28 01:00:00,0.0,67.0,69.0,90.0,0.0,29.08,571065.5,6.9625
143809,2024-05-28 02:00:00,0.0,68.0,69.0,93.0,0.0,29.07,571069.5,6.8675


In [67]:
# Let's get rid of the Unnamed column

combined_df.drop('Unnamed: 0', axis=1, inplace=True)

# set 'DATE' column as index

combined_df.set_index('DATE', inplace=True)

In [69]:
# Save it in a dataframe to be used later

combined_df.to_csv('final_data.csv')

In [71]:
combined_df

,Precip,WetBulbTemp,DryBulbTemp,RelHumidity,WindSpeed,StationPressure,gauge_height
DATE,,,,,,,
2008-01-01 01:00:00,0.0,37.0,38.0,89.0,0.0,29.19,5.6450
2008-01-01 02:00:00,0.0,38.0,39.0,89.0,5.0,29.18,5.5425
2008-01-01 03:00:00,0.0,38.0,40.0,86.0,5.0,29.18,5.4575
2008-01-01 04:00:00,0.0,38.0,40.0,86.0,6.0,29.19,5.3625
2008-01-01 05:00:00,0.0,41.0,45.0,68.0,11.0,29.21,5.2175
...,...,...,...,...,...,...,...
2024-05-27 23:00:00,0.0,68.0,73.0,79.0,3.0,29.06,7.2175
2024-05-28 00:00:00,0.0,68.0,71.0,87.0,0.0,29.07,7.0725
2024-05-28 01:00:00,0.0,67.0,69.0,90.0,0.0,29.08,6.9625
